In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [2]:
# 크롬 웹드라이버 실행 경로
path = 'chromedriver.exe'

# 크롤링할 주소 (네이버 세부설정까지 완료한 상태로)
target_url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.06.30&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1009&nso=so%3Ar%2Cp%3Afrom20230101to20230630&is_sug_officeid=0&office_category=0&service_area=0"

# 크롬 드라이버 사용
service = Service(executable_path=path) # selenium 최근 버전은 이렇게
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# 드라이버에게 크롤링할 대상 알려주기
try:
    # 웹페이지 열기
    driver.get(target_url)

    # 초기 스크롤 높이 설정
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 아래까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 적응 시간
        time.sleep(0.5)  

        # 새로운 스크롤 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 만약 더 스크롤할게 없다면 정지
        last_height = new_height

    # 해당 화면 안에서 페이지 소스 받기
    page_source = driver.page_source

    # BeautifulSoup으로 parsing
    bs_obj = BeautifulSoup(page_source, 'html.parser')

    # a태그로 info에 해당하는 태그들 다 수집
    results = bs_obj.find_all('a', {'class': "info"})

    # 수집한 a 태그 개수 -> 절반이 네이버 뉴스
    print(f"Number of results found: {len(results)/2}")

finally:
    # 가상 화면 닫기
    driver.quit()

Number of results found: 2260.0


In [3]:
# 기존 한국경제 링크랑 섞여서 네이버 뉴스꺼만 필터링
news_link = []
for tag in results:
    href = tag.get('href')
    if href:
        news_link.append(href)

news_link = [i for index, i in enumerate(news_link) if (index + 1) % 2 == 0]

In [4]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [5]:
from tqdm import tqdm

all_results = dict()

# tqdm을 사용하여 진행 상황 표시
for i, link in enumerate(tqdm(news_link, desc="Processing links")):     
    # scraping하려는 웹페이지 주소를 get()에 전달
    driver.get(link)
    all_results[i] = dict()
    
    # 일시 추출하기(연예, 스포츠 예외처리)
    try:
        date = driver.find_elements(By.CLASS_NAME, 'media_end_head_info_datestamp_bunch')
        date = date[0].text[2:]
    except (IndexError, AttributeError):
        try:
        # 첫 번째 시도가 실패했을 경우, 두 번째 클래스로 시도
            date = driver.find_elements(By.CLASS_NAME, 'date')
            date = date[0].text[2:]
        except (IndexError, AttributeError):
        # 두 번째 시도가 실패했을 경우, 세 번째 클래스로 시도
            date = driver.find_elements(By.CLASS_NAME, 'NewsEndMain_date__xjtsQ')
            date = date[0].text[2:]
    
    # 제목 추출하기(연예, 스포츠 예외처리)
    try:
        title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
        title = title[0].text
    except (IndexError, AttributeError):
        title = driver.find_elements(By.CLASS_NAME, 'NewsEndMain_article_head_title__ztaL4')
        title = title[0].text
        
    # 본문 추출하기(연예, 스포츠 예외처리)
    try:
        body = driver.find_elements(By.ID, 'newsct_article')
        body = body[0].text.replace('\n','')
    except (IndexError, AttributeError):
        body = driver.find_elements(By.CLASS_NAME, '_article_content')
        body = body[0].text.replace('\n','')
    
    # 결과 저장
    all_results[i]['link'] = link
    all_results[i]['date'] = date
    all_results[i]['title'] = title
    all_results[i]['body'] = body

# 드라이버 종료
driver.quit()

Processing links: 100%|██████████████████████████████████████████████████████████| 2260/2260 [1:07:54<00:00,  1.80s/it]


In [18]:
news_df = pd.DataFrame(all_results).T

In [19]:
news_df

,link,date,title,body
0,https://n.news.naver.com/mnews/article/009/000...,2023.01.01. 오전 8:02,“국민주? 쳐다보기도 싫다”…일년 내내 마음고생만 한 동학개미,삼성전자·카카오·네이버 등개인 보유 상위종목에 이름 올려네카오 평균수익률 -40%대...
1,https://n.news.naver.com/mnews/article/009/000...,2023.01.01. 오후 4:21,스마트폰 美·中 갈등 여파 글로벌 판매 2년 연속 12억대 '부진',글로벌 경제가 본격적으로 경기 둔화의 사정권에 들어가면서 올해 스마트폰 시장은 부진...
2,https://n.news.naver.com/mnews/article/009/000...,2023.01.01. 오후 4:21,지갑 닫는 소비자 … 초연결·개인화로 승부수,가전삼성전자가 'CES 2023'에서 공개할 프리미엄 주방 가전인 비스포크 인피니트...
3,https://n.news.naver.com/mnews/article/009/000...,2023.01.01. 오후 4:21,코로나 특수 끝나 … 메모리 보릿고개 넘을듯,반도체올해 반도체 업계는 긴축경영으로 보릿고개를 넘을 것으로 예상된다. 사진은 삼성...
4,https://n.news.naver.com/mnews/article/009/000...,2023.01.01. 오후 4:23,미래 신사업에 450조 투자 … 메모리 초격차·6G 선점나서,삼성전자삼성전자가 세계 최초로 GAA 기술을 적용한 3나노 파운드리 공정 기반의 초...
...,...,...,...,...
2255,https://n.news.naver.com/mnews/article/009/000...,2023.06.30. 오후 5:34,"[포토] 삼성전자 ""장마철에도 우리 가족 뽀송뽀송""",삼성전자가 본격적인 장마철을 맞아 7월 한 달간 세탁기·건조기 상하 일체형 '비스포...
2256,https://n.news.naver.com/mnews/article/009/000...,2023.06.30. 오후 5:42,퓨리오사·사피온·리벨리온 자금 빨아들이는 'AI반도체',국내 대표 인공지능(AI) 반도체 개발 스타트업 세 곳이 지난 1년 새 국내 대기업...
2257,https://n.news.naver.com/mnews/article/009/000...,2023.06.30. 오후 5:42,안 받던 것 받으면 짭짤할 터...애플페이로 고민 커진 삼성전자,[사진출처 = 삼성페이 공식 홈페이지]애플페이가 국내 상륙한지 100일이 지난 가운...
2258,https://n.news.naver.com/mnews/article/009/000...,2023.06.30. 오후 5:52,[단독]사피온 이어 퓨리오사AI도...인공지능 반도체에 돈 몰린다,신규 자금 유치 나선 퓨리오사AI산은·한투파 등 700억 투자금 받아사피온·리벨리온...


In [20]:
# '오전'과 '오후'를 영어로 변환
news_df['date'] = news_df['date'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# date 열을 datetime 형식으로 변환
def try_convert_date(x):
    try:
        # 첫 번째 시도: 원래 문자열을 포맷에 맞게 변환
        return pd.to_datetime(x, format='%Y.%m.%d. %p %I:%M')
    except ValueError:
        # 첫 번째 시도가 실패할 경우: 앞에 "20"을 추가하여 다시 변환 시도
        return pd.to_datetime("20" + x, format='%Y.%m.%d. %p %I:%M')

news_df['date'] = news_df['date'].apply(try_convert_date)
                                            
# year, month, day, hour, minute 열 생성
news_df['year'] = news_df['date'].dt.year
news_df['month'] = news_df['date'].dt.month
news_df['day'] = news_df['date'].dt.day
news_df['hour'] = news_df['date'].dt.hour
news_df['minute'] = news_df['date'].dt.minute

In [21]:
# 고유 인덱스 부여 (년도 + 6자리)
# 1이면 -> 000001
news_df.index = news_df.index + 1
news_df = news_df.reset_index()
news_df['index'] = news_df['index'].astype(str).str.zfill(6)
news_df['index'] = str(news_df['year'].unique()[0]) + news_df['index']

In [24]:
news_df

,index,link,date,title,body,year,month,day,hour,minute
0,2023000001,https://n.news.naver.com/mnews/article/009/000...,2023-01-01 08:02:00,“국민주? 쳐다보기도 싫다”…일년 내내 마음고생만 한 동학개미,삼성전자·카카오·네이버 등개인 보유 상위종목에 이름 올려네카오 평균수익률 -40%대...,2023,1,1,8,2
1,2023000002,https://n.news.naver.com/mnews/article/009/000...,2023-01-01 16:21:00,스마트폰 美·中 갈등 여파 글로벌 판매 2년 연속 12억대 '부진',글로벌 경제가 본격적으로 경기 둔화의 사정권에 들어가면서 올해 스마트폰 시장은 부진...,2023,1,1,16,21
2,2023000003,https://n.news.naver.com/mnews/article/009/000...,2023-01-01 16:21:00,지갑 닫는 소비자 … 초연결·개인화로 승부수,가전삼성전자가 'CES 2023'에서 공개할 프리미엄 주방 가전인 비스포크 인피니트...,2023,1,1,16,21
3,2023000004,https://n.news.naver.com/mnews/article/009/000...,2023-01-01 16:21:00,코로나 특수 끝나 … 메모리 보릿고개 넘을듯,반도체올해 반도체 업계는 긴축경영으로 보릿고개를 넘을 것으로 예상된다. 사진은 삼성...,2023,1,1,16,21
4,2023000005,https://n.news.naver.com/mnews/article/009/000...,2023-01-01 16:23:00,미래 신사업에 450조 투자 … 메모리 초격차·6G 선점나서,삼성전자삼성전자가 세계 최초로 GAA 기술을 적용한 3나노 파운드리 공정 기반의 초...,2023,1,1,16,23
...,...,...,...,...,...,...,...,...,...,...
2255,2023002256,https://n.news.naver.com/mnews/article/009/000...,2023-06-30 17:34:00,"[포토] 삼성전자 ""장마철에도 우리 가족 뽀송뽀송""",삼성전자가 본격적인 장마철을 맞아 7월 한 달간 세탁기·건조기 상하 일체형 '비스포...,2023,6,30,17,34
2256,2023002257,https://n.news.naver.com/mnews/article/009/000...,2023-06-30 17:42:00,퓨리오사·사피온·리벨리온 자금 빨아들이는 'AI반도체',국내 대표 인공지능(AI) 반도체 개발 스타트업 세 곳이 지난 1년 새 국내 대기업...,2023,6,30,17,42
2257,2023002258,https://n.news.naver.com/mnews/article/009/000...,2023-06-30 17:42:00,안 받던 것 받으면 짭짤할 터...애플페이로 고민 커진 삼성전자,[사진출처 = 삼성페이 공식 홈페이지]애플페이가 국내 상륙한지 100일이 지난 가운...,2023,6,30,17,42
2258,2023002259,https://n.news.naver.com/mnews/article/009/000...,2023-06-30 17:52:00,[단독]사피온 이어 퓨리오사AI도...인공지능 반도체에 돈 몰린다,신규 자금 유치 나선 퓨리오사AI산은·한투파 등 700억 투자금 받아사피온·리벨리온...,2023,6,30,17,52


In [25]:
news_df.to_csv('2023_매일경제_전반기.csv', index=False, encoding='utf-8-sig')